In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import lightgbm as lgb
import xgboost as xgb
import gc
%matplotlib inline

C:\Users\deepusuresh\Anaconda3\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
C:\Users\deepusuresh\Anaconda3\lib\site-packages\dask\dataframe\utils.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
C:\Users\deepusuresh\Anaconda3\lib\site-packages\distributed\config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [2]:
train = pd.read_excel('Data_Train.xlsx')
test = pd.read_excel('Data_Test.xlsx')
sub = pd.read_excel('Sample_Submission.xlsx')

In [16]:
display(train.head())
display(test.head())

,Title,Author,Edition,Synopsis,Genre,BookCategory,Price,Released_year
0,4763,615,1042,"(0, 36051)\t0.14564766861065542\n (0, 17432...",Action & Adventure (Books),Action & Adventure,220.00,2016
1,1721,307,2762,"(0, 36051)\t0.14564766861065542\n (0, 17432...",Cinema & Broadcast (Books),"Biographies, Diaries & True Accounts",202.93,2012
2,2446,3390,1960,"(0, 36051)\t0.14564766861065542\n (0, 17432...",International Relations,Humour,299.00,1982
3,155,62,2615,"(0, 36051)\t0.14564766861065542\n (0, 17432...",Contemporary Fiction (Books),"Crime, Thriller & Mystery",180.00,2017
4,2343,953,81,"(0, 36051)\t0.14564766861065542\n (0, 17432...",Photography Textbooks,"Arts, Film & Photography",965.62,2006


,Title,Author,Edition,Synopsis,Genre,BookCategory,Released_year
0,1142,1071,227,"(0, 3954)\t0.09037744484135733\n (0, 13478)...",Short Stories (Books),"Crime, Thriller & Mystery",1986
1,665,356,1166,"(0, 3954)\t0.09037744484135733\n (0, 13478)...",Operating Systems Textbooks,"Computing, Internet & Digital Media",2018
2,184,239,512,"(0, 3954)\t0.09037744484135733\n (0, 13478)...",Romance (Books),Romance,2011
3,1394,45,638,"(0, 3954)\t0.09037744484135733\n (0, 13478)...",Programming & Software Development (Books),"Computing, Internet & Digital Media",2016
4,862,933,745,"(0, 3954)\t0.09037744484135733\n (0, 13478)...",Linguistics (Books),"Language, Linguistics & Writing",2011


In [17]:
display(train.shape)
display(test.shape)

(6237, 8)

(1560, 7)

In [4]:
for df in [train,test]:
  df.pop('Reviews')
  df.pop('Ratings')

In [5]:
for df in [train,test]:
  df['Ratings_int'] = df['Ratings_int'].map(lambda x:x.split(','))
  df['Ratings_int'] = df['Ratings_int'].map(lambda x:''.join(x))
  df['Ratings_int'] = df['Ratings_int'].astype(int)
  df['Reviews_Ratings'] = df['Reviews_int']*df['Ratings_int']

KeyError: 'Ratings_int'

In [6]:
for df in [train,test]:
  df['Released_year'] = df['Edition'].map(lambda x:x.split(' ')[-1])

In [7]:
for df in [train,test]:
  df['Synopsis'] = TfidfVectorizer().fit_transform(df['Synopsis'])

In [8]:
for df in [train,test]:
  df['Reviews_int'] = df['Reviews'].map(lambda x:x.split(' ')[0])

KeyError: 'Reviews'

In [9]:
for df in [train,test]:
  df['Reviews_int'] = df['Reviews_int'].astype(float)
  df['ratio_of_review'] = df['Reviews_int']/5

KeyError: 'Reviews_int'

In [10]:
for df in [train,test]:
  df['Ratings_int'] = df['Ratings'].map(lambda x:x.split(' ')[0])

KeyError: 'Ratings'

In [11]:
le = LabelEncoder()

In [12]:
for c in test.columns:
  for df in [train,test]:
    if df[c].dtype == 'object':
      df[c] = le.fit_transform(df[c])

TypeError: unhashable type: 'csr_matrix'

In [13]:
x = train.drop('Price',axis=1)
y = train['Price']

In [14]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.01)

In [15]:
folds = KFold(n_splits=5)

lgb_fold_pred = 0
lgb_val_pred = 0

lr = 0.835
step = 0.6985

#validation_data = lgb.Dataset(validation_x,label=validation_y)

for fold, (trn_idx, test_idx) in enumerate(folds.split(x_train)):
    print('========================================================================')
    print('Training on fold {}'.format(fold + 1))
    
    trn_data = lgb.Dataset(x_train.iloc[trn_idx], label=y_train.iloc[trn_idx])
    
    val_data = lgb.Dataset(x_train.iloc[test_idx], label=y_train.iloc[test_idx])
    
    params = {
            'min_data_in_leaf': 280, 
            'num_leaves': 430, 
            'learning_rate': lr,
            'min_child_weight': 0.0227696,
            'bagging_fraction': 0.625789, 
            'feature_fraction': 0.47,
            'reg_lambda': 0.72,
            'reg_alpha': 0.46,
            'max_depth': -1, 
            'objective': 'regression',
            'seed': 59,
            'tree_learner':'serial',
            'feature_fraction_seed': 42,
            'n_jobs':-1,
            'bagging_seed': 42,
            'drop_seed': 49,
            'data_random_seed': 42,
            'boosting_type': 'goss',
            'verbose': 1,
            'metric':'rmse',
            'od_wait':500,
            }
        
    clf = lgb.train(params, trn_data, 10000, valid_sets = [trn_data, val_data], verbose_eval=200, early_stopping_rounds=50)
    
    lgb_fold_pred += clf.predict(test)
    lgb_val_pred += clf.predict(x_test)
    
    lr = lr * step**((fold/5))
    step = -1*step*np.log(0.8)
    
    del trn_data,val_data
    gc.collect()
    
lgb_val_pred /= 5
lgb_fold_pred /= 5

Training on fold 1


ValueError: DataFrame.dtypes for data must be int, float or bool.
Did not expect the data types in the following fields: Synopsis, Genre, BookCategory, Released_year

In [176]:
lgb_val_pred

array([505.71042438, 424.58776232, 599.7508568 , 350.17152488,
       439.93935817, 471.38513627, 772.15070301, 426.04908698,
       488.28402158, 430.6081608 , 693.74020701, 484.44426437,
       743.17184835, 570.9561335 , 469.42347335, 552.72952224,
       610.28776849, 576.22799353, 482.39062387, 551.13190586,
       422.13507696, 636.02349483, 702.13489604, 406.6469155 ,
       440.84303513, 904.72217525, 423.19155114, 417.65489838,
       336.87816956, 560.55657203, 455.80440219, 646.94500758,
       466.558947  , 499.63942832, 531.94677759, 644.09588777,
       607.42435126, 438.80459107, 445.55880524, 558.47870557,
       482.02252217, 724.47929156, 735.5620427 , 581.50298668,
       594.88920627, 670.03407031, 579.47669796, 435.13669546,
       393.00395024, 456.01050621, 415.55505847, 567.48939739,
       453.49336163, 661.39081245, 660.35060109, 416.63853811,
       783.84264276, 641.22347201, 454.14804868, 724.13336922,
       418.58028973, 472.83679355, 423.07864558])

In [0]:
sub['Price'] = lgb_fold_pred
sub.to_excel('book_prediction.xlsx',index=False)